In [27]:
import os
os.getcwd()

os.listdir()


['01_preprocessing.ipynb',
 '03_training_detector.ipynb',
 '06_train_classifier.ipynb',
 '02_augmentation_detection.ipynb',
 'yolov8n.pt',
 'work_dirs',
 'rtmdet_tiny_carDD.py',
 '04_build_classifier_dataset.ipynb',
 'runs_carDD',
 '05_augmentation_classifier.ipynb']

In [28]:
from ultralytics import YOLO

# load pretrained YOLOv8n for detection
model = YOLO("yolov8n.pt")

# train
model.train(
    data="/Users/prisriva/Desktop/fai_final/damage.yaml",   # relative path from notebook directory
    epochs=30,
    imgsz=640,
    batch=4,
    project="runs_carDD",
    name="detector_model"
)


Ultralytics 8.3.232 🚀 Python-3.10.19 torch-2.9.1 CPU (Apple M4)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/Users/prisriva/Desktop/fai_final/damage.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=detector_model3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, 

RuntimeError: Dataset '/Users/prisriva/Desktop/fai_final/damage.yaml' error ❌ Dataset '/Users/prisriva/Desktop/fai_final/damage.yaml' images not found, missing path '/Users/prisriva/Desktop/fai_final/datasets/data/val'
Note dataset download directory is '/Users/prisriva/Desktop/fai_final/datasets'. You can update this in '/Users/prisriva/Library/Application Support/Ultralytics/settings.json'

In [20]:
"""
Basic imports and environment verification.
Ensures MMDetection, MMCV, and PyTorch are correctly installed.
"""

import torch
import mmcv
import mmdet
from mmengine.runner import Runner

print("Torch:", torch.__version__)
print("MMCV:", mmcv.__version__)
print("MMDet:", mmdet.__version__)


Torch: 2.9.1
MMCV: 2.1.0
MMDet: 3.3.0


In [21]:
DATA_ROOT = "./images/"       # folder containing all images
TRAIN_ANN = "train.json"      # COCO annotation file
VAL_ANN = "val.json"          # COCO annotation file
CONFIG_FILE = "rtmdet_tiny_carDD.py"
WORK_DIR = "./work_dirs/rtmdet_tiny"

In [22]:
"""
Creates a minimal RTMDet-tiny config that MMDetection will use.
This config points to our CarDD dataset and adjusts the number of classes.
"""

config_text = """
_base_ = 'mmdet::rtmdet/rtmdet_tiny_8xb32-300e_coco.py'

model = dict(
    bbox_head=dict(num_classes=1)
)

train_dataloader = dict(
    dataset=dict(
        type='CocoDataset',
        data_root='./',
        ann_file='train.json',
        data_prefix=dict(img='images/'),
    )
)

val_dataloader = dict(
    dataset=dict(
        type='CocoDataset',
        data_root='./',
        ann_file='val.json',
        data_prefix=dict(img='images/'),
    )
)

default_hooks = dict(
    checkpoint=dict(
        interval=1,
        max_keep_ckpts=3
    )
)

work_dir = './work_dirs/rtmdet_tiny'
"""

with open(CONFIG_FILE, "w") as f:
    f.write(config_text)

print(f"Config saved → {CONFIG_FILE}")


Config saved → rtmdet_tiny_carDD.py


In [23]:
"""
Constructs the MMEngine training Runner using our RTMDet config.
This loads the config file we just created.
"""

from mmengine import Config

cfg = Config.fromfile(CONFIG_FILE)

runner = Runner.from_cfg(cfg)
runner


/Users/prisriva/miniconda3/envs/cardd/lib/python3.10/importlib/__init__.py:169: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  _bootstrap._exec(spec, module)


ModuleNotFoundError: No module named 'mmcv._ext'